In [1]:
##AWS 람다 테스트용 코드


import requests
url = "https://xxanatchmvujwbl7cz2nguy3440hgaoq.lambda-url.ap-northeast-2.on.aws/"
r = requests.post(url, json={"issue_key": "P2-70735", "update": True})
print(r.status_code, r.text)

200 {"issue_key":"P2-70735","update_payload":{"summary":"[Research] Hydraulics 리서치 노드의 연구 보상이 기획 의도와 다르게 적용되어 있는 현상 \/ The research reward of the Hydraulics research node is being applied differently than intended by the design","description":"Observed\/관찰 결과:\nHydraulics 리서치 노드의 연구 보상이 기획 의도와 다르게 적용되어 있는 현상이 발생합니다.\n{color:#4c9aff}An issue occurs where the research reward of the Hydraulics research node is being applied differently than intended.{color}\n\n* 기획 문서를 참고하면 Hydraulics 리서치 노드의 연구 보상은 Salvage recipe: Hydraulic Cylinder로 적용되어야 하지만, 실제론 Salvage recipe: Jackhammer로 지급됩니다.\n{color:#4c9aff}According to the design document, the research reward for the Hydraulics research node should be Salvage recipe: Hydraulic Cylinder, but in reality it is granted as Salvage recipe: Jackhammer.{color}\n* Hydraulics의 Unlocks, Unlocked 영역에 연구 보상 명칭은 Salvage recipe: Hydraulic Cylinder로 출력되지만, 아이콘은 Jackhammer로 출력됩니다.\n{color:#4c9aff}In Hydraulics' Unlocks \/ Unlocked sections the research reward na